In [11]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
 
# sentences = MySentences('stanfordSentimentTreebank/datasetSentences.txt') # a memory-friendly iterator
with open('stanfordSentimentTreebank/datasetSentences.txt') as f:
    sentences = f.read().splitlines()
model = gensim.models.Word2Vec(sentences)
print(sentences[1000])
model = gensim.models.Word2Vec() # an empty model, no training
# model.build_vocab(some_sentences)  # can be a non-repeatable, 1-pass generator
# model.train(other_sentences)  # can be a non-repeatable, 1-pass generator

1000	It is not a mass-market entertainment but an uncompromising attempt by one artist to think about another .
